# Who owns Lausanne? 

## Installation

In order to run this notebook you will need multiple dependecies. We assume you have a running `conda` distribution.

 - `numpy` needs to be in version `>=0.15`
 - The JSON processing tool `jq` is needed. Install with your OS' package manager (e.g. `brew install jq`).
 - `pip install yq geopy shapely geopandas`
 - If you want to have a look at the raw geographical data you have to intall [QGIS](https://www.qgis.org/en/site/), an open source geo information system tool.

_If you are running an older version of macOS (e.g. 10.11) you might need to call `ulimit -n 1024` in the terminal before starting `jupyter`. This can avoid a bug with one of the preprocessing scripts._

## Imports

We start importing some core python libraries that will be used throughout the whole Jupyter Notebook.

In [ ]:
import numpy as np
import json

import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(font="Helvetica Neue")
themecolors = sns.color_palette(["#5B698C","#84CFAC", "#BD8273","#0E285C",  "#B2A7DB"])
sns.set_palette(themecolors)

sns.set_style("whitegrid", {
    "axes.edgecolor": "0.9",
    "grid.color": "0.9"
})

# Auto reload module
%load_ext autoreload
%autoreload 2

# Suppress future warning (generated by seaborn)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import sklearn

## 1. Public data and owners

We obtained ftp access from the Lausanne office of cadastre. The data is a collection of ESRI shapefiles, describing roads, buildings, parcels, trees, waterbodies, and others.
Each shapefile is a collection of features, and each feature has an associated geometry (e.g. the shape of a land parcel) and associated attributes ( e.g. the commune responsible for the parcel, the parcel number).

We can explore this dataset by using GIS software that supports shapefiles. We used QGIS to explore the dataset.
We hoped to find an attribute describing the parcel owner in the parcel shapefile layer, but it wasn't there.
We had to resort to web scraping to recover this attribute.

## 1.1 Scraping owners

### 1.1.2 Download XML files
We wanted to associate each parcel in Lausanne to an owner. To do this, we divided Lausanne's surface in rectangles, and requested parcel informations for these rectangles to a service exposing the owners name.
The code for the scraping is in [`/scraping/owners/scrape_owners_to_xml.py`](/edit/scraping/owners/scrape_owners_to_xml.py).

The result of owner scraping is a set of 400 xml files, each containing parcel information for a geographical rectangle. The data are saved in the following directory: `data/raw/owners/`.

For privacy reason we decided not to push any data on the online github respository.

We start exploring the raw owner xml data:

In [ ]:
!ls "data/raw/owners/" | wc -l

Each file is named after the coordinates, in the Swiss systems, of the top-left and bottom-right points bounding the scraped rectangle.

In [ ]:
!ls "data/raw/owners/" 2>/dev/null | head -3 # suppress error message by redirecting errors to null

One example file:

In [ ]:
!head -n 10 "data/raw/owners/534810.4210526316_155847.0_535161.3710526315_155589.0.xml"

### 1.1.2 From XML to a single JSON
We use `yq`, `xq` and `jq` programs to extract only the features we care about from the different XML files and save them as a list of objects in a single json file.
[`scraping/owners/multiple_xml_to_single_json.sh`](/edit/scraping/owners/multiple_xml_to_single_json.sh) is a small script leveraging the expressiveness of `jq` to efficiently concatenate the XML files into a single json, while also discarding all the attributes we have no interest in.

The result file of the shell program is a JSON file`/data/owners/all_owners_dirty.json`.

In [ ]:
# Convert all XML into one JSON file
!scraping/owners/multiple_xml_to_single_json.sh all_owners_dirty.json

If you see errors of the form
```
jq: error (at <stdin>:1): Cannot iterate over null (null)
```
you don't have to worry. These are just errors, when `jq` encounters the end of a file. The script is still working correctly.

### 1.1.3 Remove duplicated and  clean owners JSON
The generated `all_owners_dirty.json` JSON has duplicate entries, entries concerning other communes than Lausanne and entries with missing owners. Furthermore, JSON is not the best format to handle tabular data. The code in [`scraping/owners/owner_json_to_clean_csv.py`](/edit/scraping/owners/owner_json_to_clean_csv.py) cleans the duplicates and tranfsorms the data into `all_owners.csv` CSV file.

In [ ]:
# Clean dirty data and write it to 'all_owners.csv' 
import scraping.owners
scraping.owners.owner_json_to_clean_csv.main(
    './data/owners/all_owners_dirty.json',
    './data/owners/all_owners.csv'
)

For the sake of semplicity, we can now remove the legacy file `all_owners_dirty.json`

In [ ]:
! rm ./data/owners/all_owners_dirty.json

### 1.1.4 Joining the owners data with the cadastre shapefiles
The result of the previous preprocessing steps is a CSV file with three columns: commune number, parcel number, and the owner name:

In [ ]:
import pandas as pd
all_owners = pd.read_csv('data/owners/all_owners.csv')
all_owners.head(3)

## 1.2 Cadastral data - data exploration

We would like to add the owner name to the attributes of the parcels shapefile that we obtained from "Office du Cadastre". To do so, we import the `all_owners.csv` csv and the shapefile in *QGIS*, and we join this two "tables" by parcel number. The resulting geographical layer contains all the geographical features representing the parcels, and additionally the owner name for each parcel. 

We can now export this layer as a GeoJSON, making sure to use `WGS-84` as the coordinate system, and continue our exploration.

The exported *geojson* file is saved at `data/owners/all_owners_parcelles.geojson`.

In [ ]:
import geopandas
all_owners_parcelles_pd = geopandas.read_file('./data/owners/all_owners_parcelles.geojson')

# Drop not used columns and rename columns
all_owners_parcelles_pd.drop(['ID_GO', 'TYPE', 'EGRID', 'GID_OLD', 'NUMCOM'], axis='columns', inplace=True)
all_owners_parcelles_pd.rename({'NO_PARC': 'parc_num', 'proprio': 'owner', 'NOM_COM':'commune'}, 
                               axis='columns', inplace=True)

all_owners_parcelles_pd.head(3)

Each row represent a parcell and each parcel has a unique owner and a *polygon* object (geometry) that represent the surface area.

In [ ]:
len(all_owners_parcelles_pd)

### 1.2.1 Who is the biggest real estate owners?

We can now quickly answer questions such as who are the 30 biggest property owners in Lausanne, by using the number of parcels owned as a measure:

In [ ]:
parcels_per_owner = all_owners_parcelles_pd['owner'].value_counts()
parcels_per_owner.head(30)

We can see that most of the biggest owners are either corporations, pension funds, or public institutions. 

We note that there is no a single private person in the list. This data tell us that our analysis will have to take into consideration other kind of owners than the private ones.

In [ ]:
np.sum(all_owners_parcelles_pd['owner'] == 'Commune de Lausanne') / len (all_owners_parcelles_pd) * 100

We also records that the Lausanne municipality owns ~12% of properties in Lausanne.

### 1.2.2 Unique owners

After dropping the non assigned values, we can see both the total number of parcels owned and the unique owners:

In [ ]:
owners = all_owners_parcelles_pd['owner'].dropna()
print('Total parcels', len(owners))
print('Unique owners', len(owners.unique()))

There are almost 8'000 parcels in Lausanne. 

We are interestd to know how many people and societies own them. This number doesn't account for PPE (_propriété par étage_, single flats owned by privates). A lower bound on the owners can be estimated by discarding the PPE entries altogether:

In [ ]:
lower_bounds_owners = len(owners[~owners.str.contains('PPE ')].unique())
print('Unique lower-bounds owners', lower_bounds_owners)

This is a lower bound for the number of owners. Although, the real number is likely to be much higher since it's unprobable that most of these unique owners are also owners of a PPE share.

### 1.2.3 Visualizing the distribution of missing values

In [ ]:
# Compute portion of missing values
all_owners_parcelles_pd['owner'].isna().mean()

22% of the parcels don't have owner information. Indeed, many parcels reperesent roads, and as such they didn't have an owner on the site we scraped. Also we didn't scrape the values for the northern part of Lausanne, which is mostly farmland and woods.

Let's try to visualize the missing values on a map:

In [ ]:
import folium
!mkdir export

In [ ]:
from heatmap import missing_values
m = missing_values(all_owners_parcelles_pd._to_geo())
m.save('export/missing_values.html')
m

The resulting map is accessible [here](export/missing_values.html)

This visualization is not very snappy or legible, but we can interpret it as follows:

- Red areas are parcels for which the proprietary is not assigned, i.e `None`. The northern parts of Lausanne were not scraped, since we didn't want to overload the scraped website and since they're mostly rural areas. It is expected that they are red.
- Zooming into central Lausanne, we see that roads have unknown owners. This is also expected.
- For some areas blue and red overlap, yielding purple parcels. This is because the dataset is slightly dirty and some bigger parcels with no owners _contains_ smaller parcels with known owners. Therefore the colors overlap.

Having asserted that the dataset is fairly sane, we can drop the features were the owner is `np.nan`, since they will be of no use to us (roads), and will make the map drawing slower.

How many parcelles have no owner?

In [ ]:
np.sum(all_owners_parcelles_pd['owner'].isna())

We can now select only the parcelles for which there is a owner, i.e drop not assigned rows to our dataframe: 

In [ ]:
complete_owners_parcelles_pd = all_owners_parcelles_pd.dropna().copy()
len(complete_owners_parcelles_pd)

From now on, our analysis will be based on such dataframe: `complete_owners_parcelles_pd`

In [ ]:
complete_owners_parcelles_pd.head(2)

###  1.2.4 Show parcels by owner type

The parcel owner format allows us to know the category of each owner. 
We will use similar categories as the statistical office of the city of Lausanne [here](https://www.lausanne.ch/officiel/statistique/quartiers/tableaux-donnees.html):

- privates
- public institutions
- companies (corporations)
- cooperatives
- pension funds
- foundations
- PPE

Societies are detected by having 'AG' or 'SA' in their name. Similary for cooperatives, foundations, and pension funds.

In [ ]:
import re
def categorize(owner):
    """Given a owner string, return category of such owner"""
    regex_cats = [
            ('retraites|pension|prévoyance|prevoyance|BVK|'+\
             'anlagestiftung|fondation d\'investissement|fondation de placement|'+\
             'vorsorge|anlage stiftung', 'pension'),
            ('commune de lausanne|dfire|cff|domaine public|Etat de Vaud|Service du logement', 'public'),
            (r's\.a\.|\bsa\b|\bag\b|société anonyme|sàrl|\bBCV\b|SICAV', 'société'), # BCV société ou public?
            (r'fondation|\bstiftung|foundation|association|fédération', 'fondation/association'),
            (r'\bppe\b|copropriété|copropriete|parcelles', 'PPE'),
            ('société coopérative|societe cooperative', 'coop'),
            ('.*', 'private')
    ]
    for cat_re, category in regex_cats: 
        if re.search(cat_re, owner, flags=re.IGNORECASE):
            return category

In [ ]:
complete_owners_parcelles_pd['owner_type'] = complete_owners_parcelles_pd['owner'].apply(lambda r:
                                                                                        categorize(r))
complete_owners_parcelles_pd.head(2)

We can visually see with a map the distribution of owners by category.

In [ ]:
from heatmap import by_owners_category
parcels_categories = complete_owners_parcelles_pd[['parc_num', 'owner_type']]
parcels_categories = parcels_categories.set_index('parc_num')
m = by_owners_category(complete_owners_parcelles_pd._to_geo(), parcels_categories )
m.save("export/by_owners_category.html")
m

The resulting map is accessible [here](export/by_owners_category.html), the different owners catagories are:

<p> <strong style='color:#b3de69'>public institutions: </strong><em>the city, the swiss railways etc.</em></p>

<p><strong style='color:#bebada'>pension or similar funds: </strong><em>investment foundations,the city’s
pension fund, etc.</em></p>

<p><strong style='color:#fb8072'>corporations: </strong><em>listed public companies like Swiss Life S.A., Régie
  Chamot &amp; Cie S.A. etc.</em></p>

<p><strong style='color:#ffe300'>cooperatives: </strong><em>registered cooperative companies like Migros, la
  Mobilière etc.</em></p>

<p><strong style='color:#80b1d3'>foundations and associations: </strong><em>for example the olympic foundation for
 cultural heritage.</em></p>

<p><strong style='color:#fdb462'>PPE: </strong><em>the single flats in a building are owned by different people
 – in french</em> proriété par étage.</p>

<p><strong style='color:#8dd3c7'>individual privates: </strong><em>the whole building is owned by a private
 citizen.</em></p>

## 2. Rents data

### 2.1 Scraping

In order to analyse how ownership patterns influence prices, we needed to complement the owners dataset with rent prices.
Rent prices are generally not public, but we can scrape from real estate websites' current rent listings, and extract the prices from there.

We scraped from [anibis.ch](https://www.anibis.ch/fr/default.aspx), [homegate.ch](https://www.homegate.ch/fr) and [tutti.ch](https://tutti.ch) and extracted up-to-date real estate announcements.

#### 2.1.1 Download the raw rents data

The scripts to download the data from the three portals are the following:

- for homegate: [`scraping/homegate/scrape_homegate.py`](/edit/scraping/homegate/scrape_homegate.py)

- for anibis:
    1. [`scraping/anibis/anibis_scrape_listings.py`](/edit/scraping/anibis/anibis_scrape_listings.py) to download the index of results matching rents in lausanne
    2. [`scraping/anibis/anibis_scrape_offers.py`](/edit/scraping/anibis/anibis_scrape_offers.py) to download each single rent offer, given a parsed index

- for tutti:
    1. [`scraping/tutti/tutti_scrape_listings.py`](/edit/scraping/tutti/tutti_scrape_listings.py) to download the index of results matching rents in lausanne
    
 
The raw rents data are then saved in `data/raw`.

#### 2.1.2 Parse the rents data


Once downloaded, we parse the data in an agreed JSON format.

#### Tutti

In [ ]:
from scraping.tutti import tutti_parse_listings
tutti_parse_listings.main()

#### Anibis

1. [`scraping/anibis/anibis_parse_listings.py`](/edit/scraping/anibis/anibis_parse_listings.py) to parse the listings index.
2. [`scraping/anibis/anibis_parse_offers.py`](/edit/scraping/anibis/anibis_parse_offers.py) to parse the pages for each offer.

### 2.2 Removing duplicates

Most rent listings are published in several websites. When merging the data sources, we first need to figure out which results are present in multiple datasets to avoid duplicate datapoints. We consider listings to be duplicates if they have the same address and the same price.

In [ ]:
!jq length data/raw/rents/tutti.json
!jq length data/raw/rents/anibis_with_streets.json
!jq length data/raw/rents/homegate.json

Before cleaning and merging we have a total of ~1300 offers.

The parsed files have this structure:

In [ ]:
pd_tutti = pd.read_json("./data/raw/rents/tutti.json")
pd_tutti.head(2)

In addition to removing duplicates, [`cleaning/merge_rent_offers.py`](/edit/cleaning/merge_rent_offers.py) clean offers without prices, without addresses, or without surface area. 

The merge script return a JSON file with all listings, one each line.

In [ ]:
import cleaning as cleaning
filenames = ["./data/raw/rents/tutti.json", 
             "./data/raw/rents/anibis_with_streets.json",
            "./data/raw/rents/homegate.json"]
dirty_rent_prices = cleaning.merge_rent_offers.main(filenames)
len(dirty_rent_prices)

### 2.3 Mapping street addresses to coordinates

In [ ]:
dirty_rent_prices_pd = pd.DataFrame.from_dict(dirty_rent_prices)
dirty_rent_prices_pd.head(2)

From the output above we can see that the address is in a textual form. 

Also, it is clear that there will be more cleaning needed. The first address is a phone number instead of an actual address. This sanitisation is automatically provided by the next script.

To perform geographical queries on addresses, we need to convert them to coordinates. To do so, we use the cadastral layer of building addresses, provided by the Cadastral office of Lausanne.
During merging of the three datasets, the addresses were standardized to use the format used by this cadastral layer.

To map an address to a coordinates couple, we iterate over all buildings in Lausanne, and check if the street name and the street number match those of our address. If there's a match, we extract the coordinates of the building from the cadastral layer. If there isn't we drop the offer (like the phone number above) and therefore perform some cleaning.

We use [`cleaning/address_to_coords.py`](/edit/cleaning/address_to_coords.py) to execute the operations descripted above.

In [ ]:
rent_prices = cleaning.address_to_coords.main(dirty_rent_prices)
rent_prices_pd = pd.DataFrame.from_dict(rent_prices)
# dropn not used columns
rent_prices_pd.drop(['city', 'meuble'], axis='columns', inplace=True)

# Explode from 'position' column latitude and longitude
long_lat = rent_prices_pd['position'].apply(lambda pos: pd.Series(pos))
long_lat.columns = ['long', 'lat']
long_lat.head(2)
rent_prices_pd =  pd.concat([rent_prices_pd, long_lat], axis='columns')

# Convert surface type to float
rent_prices_pd['surface'] = rent_prices_pd['surface'].astype(float)
rent_prices_pd.head(2)

In [ ]:
len(rent_prices_pd)

The resulting *Pandas* DataFrame `rent_prices_pd` if finnaly cleaned. We can now start visualizing and intrpret the dataset.

We also notice that during the cleaning phase we went from than 1300 rents announcements to 469. 

### 2.4 Visualizing the rents dataset

#### 2.4.1 Rent position

For a better spatial understanding, we will increment on some of the maps the quartiers of Lausanne. We start loading the quartiers on a *geopandas* DataFrame:

In [ ]:
quartiers_pd = geopandas.read_file('data/raw/maps/quartiers.geojson')
quartiers_pd.head(2)

In [ ]:
from heatmap import marker_rents_with_quartiers
m = marker_rents_with_quartiers(rent_prices_pd, quartiers_pd._to_geo() )
m.save("export/marker_rents_with_quartiers.html")
m

The resulting map is accessible [here](export/marker_rents_with_quartiers.html)

#### 2.4.2 Rent position by quartier

We want to map each offer to a quartier. Each rent data-point has a pair of coordinates localizing it in space. We use the python library `shapely`, that allows us to perform geometrical queries, to find the _quartier_ for each rent offer.

We start defining a function `get_quartier()` that takes as input longitudine and latitude of the queried place.

In [ ]:
# import the two data structures needed
from shapely.geometry import Point, Polygon

quartiers = quartiers_pd._to_geo()
def get_quartier(lat, long):
    for quartier in quartiers['features']:
        # skip because we don't have owner data for forest areas
        if quartier['properties']['Name'] == '90 - Zones foraines':  
            return None

        """Given longitute and latitude, return quartier in Lausanne"""
        pos = Point(long, lat)
        quartier_vertices = [(east, north) for east, north, z in quartier['geometry']['coordinates'][0]]
        quartier_poly = Polygon(quartier_vertices)
        if quartier_poly.contains(pos):
            return quartier['properties']['Name']

In [ ]:
rent_prices_pd.head(2)

We now iterate over all rents and found the respective quartier.

In [ ]:
rent_prices_pd['quartier'] = \
    rent_prices_pd[['lat', 'long']] \
    .apply(lambda row: get_quartier(row['lat'], row['long']), axis='columns')
rent_prices_pd.head(2)

Let's sanity check by changing the color of the marker depending on the found _quartier_ and displaying all of it on a map.

In [ ]:
from heatmap import circles_rents
m = circles_rents(rent_prices_pd, quartiers_pd._to_geo() )
m.save("export/circle_rents.html")
m

The resulting map is accessible [here](export/circle_rents.html)

#### 2.4.3 Rent prices by CHF/M^2

We are interested in the prices of rents by surface area:

In [ ]:
rent_prices_pd['CHF/m2'] = rent_prices_pd[['price', 'surface']].apply(lambda row:
                                                                      float(row['price'])/float(row['surface']), 
                                                                      axis='columns')
rent_prices_pd.head(2)

In [ ]:
from heatmap import circles_prices
m = circles_prices(rent_prices_pd)
m.save('export/circle_prices.html')
m

The resulting map is accessible [here](export/circle_prices.html)

We can already see that in general real estates close to the center and/or close to the lake are more expensive.

We can now see the median price by quartiers:

In [ ]:
# Display median price per neighborhood
rent_prices_pd[['CHF/m2', 'quartier']].groupby('quartier').agg('median')

As aleardy noticed on the maps, the center is the expensive quartier of Lausanne. Later we will come back to that facts and enhance our analysis. 

### 2.5 Map each offer to an owner

What about merging the two dataset that we have (`rent_prices_pd` and `complete_owners_parcelles_pd`) ?

In [ ]:
complete_owners_parcelles_pd.head(2)

Each parcel has an owner and a geometry, which is a `MultiPolygon`. A `MultiPolygon` is a list of `Polygon`s. Every `Polygon` is a list of "linear rings". The first linear ring defines the outer perimeter of the polygon, and the next linear rings define holes in the polygon.
For all parcels the multipolygons are made of only 1 polygon, and every polygon only has the outer perimeter and no holes. We can use shapely polygons again to find wether an offer is within a polygon.

We define, _get_owner_ that given a position, it return the name of the propretary of such parcelles.

In [ ]:
def get_owner(pos):
    """Reutn owner name given longitude and latitude coordinates"""
    pos = Point(pos)
    parcelle = complete_owners_parcelles_pd[complete_owners_parcelles_pd['geometry'].contains(pos)]
    owner = np.nan
    if parcelle['owner'].values.size > 0:
        owner = parcelle['owner'].values[0]
    return owner
get_owner(rent_prices_pd['position'][0])

We can therefore iterate over all *rent prices* and for each values find the respective owner. 

In [ ]:
rent_prices_pd.shape

In [ ]:
rent_prices_pd['owner'] = rent_prices_pd['position'].apply(lambda pos: get_owner(pos))
rent_prices_pd.head(2)

## 3. Interpretation of data

After a first hand-on on the data, it's time to use some models to answer some open questions. 

### 3.1 Linear model describing relation of ownership and price

As our main goal is to understand the rent price composition, we will perform linear regression on the rent prices. More precisely, we will try to predict the prices of rents in each quartier based on the features:

 - ownership proportion of each ownership type: ($f_{public}, f_{s.a.}, $...)
 - distance from the centre of the city: $dist$
 - the mean price of rents in the _quartier_ $q$ (dependent variable): $price(q)$ 
 
The linear model is then:

$$
price(q) = \beta_1 f_{public}(q) + \beta_2  f_{s.a.}(q) + ~...~ + \beta_j  f_{privates}(q)+  \beta_k dist(q)
$$

We will apply linear regression to this model and extract the knowledge from the parameters $\beta$. One problem could however be, that the ownership pattern itself depends on the distance or vice-versa. In that case we'll be able to check this assumption by predicting the distance from the ownership types:

$$
dist(q) = \beta_1 f_{public}(q) + \beta_2  f_{s.a.}(q) + ~...~ +\beta_j  f_{privates}(q)
$$


Now, each rents have it's own owner. We can iterate and add a columns _owner_type:

In [ ]:
# Remove not assigned owner
rent_prices_with_owners_pd = rent_prices_pd[~rent_prices_pd['owner'].isna()].copy()

rent_prices_with_owners_pd['owner_type'] = rent_prices_with_owners_pd['owner'].apply(lambda r: categorize(r))
rent_prices_with_owners_pd.head(2)

In [ ]:
from sklearn import linear_model
import scipy

covariates = rent_prices_with_owners_pd[['owner_type','CHF/m2']].copy()
covariates["owner_type"] = covariates["owner_type"].astype("category")
samples_per_cat = covariates["owner_type"].value_counts()

covariates = pd.get_dummies(covariates)

# drop one indicator to avoid multiple colinearity
covariates = covariates.drop("owner_type_private", axis="columns")

coeffs = np.empty(covariates.shape)

# columns: num of covariates minus the response (y) plus 1 for the intercept

# bootstrap confidence interval for linear regression coefficients
for i in range(covariates.shape[0]):
    sample = covariates.values[
        np.random.choice(
            covariates.shape[0], size=covariates.shape[0], replace=True
        )
    ]
    lm = linear_model.LinearRegression(fit_intercept=True)
    X = sample[:, 1:]
    y = sample[:, 0]
    lm.fit(X, y)
    coeffs[i, 0] = lm.intercept_
    coeffs[i, 1:] = lm.coef_

alpha = 0.05
lower, upper = np.percentile(coeffs, q=((alpha/2)*100, (1 - alpha/2)*100), axis=0)

print("%-40s\t%s\t%s\t%s" % ("feature", "%.3f qtile" % (alpha/2), "%.3f qtile" % (1 - alpha/2), "n"))
print()
print("%-40s:\t%f\t%f\t%d" % ("intercept (average for private owner)",
                              lower[0], upper[0],samples_per_cat['private']))

for typ, lower_q, upper_q in zip(covariates.columns[1:], lower[1:], upper[1:]):
    print(
        "%-40s:\t%f\t%f\t%d"
        % (typ, lower_q, upper_q, samples_per_cat[typ.split("_")[-1]])
    )


The table shows the confidence interval for each parameter of the linear model.
The `intercept` value represents the average price of the 'private' owner category, and the `owner_type_*` rows show the confidence intervals for the weights corresponding to the presence of any of the categorical variables relating to the owner type.
Every the confidence intervals for the weights corresponding to every categories contain the value $0$. This leads us to be unable to statistically say that the type of owner influences the price of the apartment

### 3.2 Linear regression on distance

We ask now if there is another relationship that influence rent prices: distance. We try to see the relation price/distance from the port of [_Ouchy_](https://map.geo.admin.ch/?lang=en&topic=ech&bgLayer=ch.swisstopo.pixelkarte-farbe&layers=ch.swisstopo.zeitreihen,ch.bfs.gebaeude_wohnungs_register,ch.bav.haltestellen-oev,ch.swisstopo.swisstlm3d-wanderwege&layers_visibility=false,false,false,false&layers_timestamp=18641231,,,&E=2537733&N=1150883&zoom=7.498594761554026&crosshair=marker).

In order to calculate distances with WGS-84 coordinates we use `geopy.great_circle`.

In [ ]:
from geopy.distance import great_circle

ouchy = 46.50766, 6.62758

rent_distances = rent_prices_pd[[
    'CHF/m2', 
    'lat',
    'long', 
    'surface',
    'quartier'
]].copy().dropna()

distances = rent_distances.apply(
    lambda row: great_circle((row.lat, row.long), ouchy).km, axis=1
)

rent_distances["distance"] = distances # distance fom Ouchy port
rent_distances.head(2)

In [ ]:
sns.lmplot(x="distance", y="CHF/m2", data=rent_distances);
plt.title("Distance from the station vs. rent prices");

This looks like something, but we need to check the significance of the model.

In [ ]:
distance_model = scipy.stats.linregress(
    rent_distances.distance, rent_distances["CHF/m2"]
)

def print_model(model):
    for stat in ["intercept", "slope", "stderr", "pvalue"]:
        print(stat, ": ", getattr(model, stat))

print_model(distance_model)

The *p-value* for the model is small enough to reject the null hypothesis of no relationship.

In [ ]:
lat_model = scipy.stats.linregress(
    rent_distances.lat ** 2, rent_distances["CHF/m2"]
)
long_model = scipy.stats.linregress(
    rent_distances.long ** 2, rent_distances["CHF/m2"]
)

print_model(lat_model)
print()
print_model(long_model)

This second analysis shows that it is really the distance to the lake. The latitude model (the higher the latitude the further away from the lake) yields a significant answer of a negative slope.

The same is not true for the longitudal model.

We check now a possible second influence, namely the surface of a flat on its price per square meter.

In [ ]:
print_model(
    scipy.stats.linregress(
        rent_distances.surface, rent_distances["CHF/m2"]
    )
)

This is maybe unexpected but the p-value is the lowest that we encoutered in the entire analysis. It seems like rent prices don't exactly grow linearly with the surface of a flat. The square meter gets cheaper for large flats, most probably because fix costs like installing a kitchen and having a parking lot stay the same for all sizes of flats.

We finally plot the two factors (distance and surface) in a pair plot.

In [ ]:
plot_df = rent_distances[["distance", "surface", "CHF/m2"]]
plot_df.columns = [
    "distance from the port Ouchy in km", 
    "surface of the flat", 
    "rent price in CHF/m^2"
]

pair = sns.pairplot(
    plot_df, 
    x_vars=["distance from the port Ouchy in km", "surface of the flat"], 
    y_vars=["rent price in CHF/m^2"],
    height=6,
    aspect=1,
    kind="reg"
);
pair.fig.suptitle("Rent price relationships");
plt.savefig("export/distance.svg", format="svg") # export in SVG to be compatible with the website

### 3.3 Predict prices for all parcelles 

We want now find the prices (CHF/m2) of rents for all parcels. To do that we use the *k-nearest-neighbor* ML-algorithm provided by *sklearn*.

In [ ]:
# Prepare the data to be trained
train = rent_prices_pd[['CHF/m2', 'long', 'lat']].copy()
train.rename({'CHF/m2':'target'},axis='columns',inplace=True)
train.head(2)

In [ ]:
# For each parcelle, find longitude and latitude, consideting the center of the Polygons
all_owners_parcelles_pd['long'] = all_owners_parcelles_pd['geometry'].apply(lambda poly: poly.centroid.x)
all_owners_parcelles_pd['lat'] = all_owners_parcelles_pd['geometry'].apply(lambda poly: poly.centroid.y)
predict = all_owners_parcelles_pd[['parc_num', 'long', 'lat']].copy()
predict.head(2)

We predict parcelles prices using `model_price_knn` method defined in [`machine_learning.py`](/edit/machine_learning.py)

In [ ]:
# Predict prices
from machine_learning import model_price_knn

predicted_chfm2, rmse, ks = model_price_knn(train, predict, np.arange(1, 50))

pd.DataFrame(rmse, ks).plot(legend=False);
plt.xlabel("k");
plt.ylabel("rmse");

The plot show the *root mean square error* for different hyperparameters K. Once the best K is found, the K-NN algorithm is executed again on all data to find the most realistic result.

We can now export the maps with all prices by parcelles.

In [ ]:
all_owners_parcelles_pd['CHF/m2'] = predicted_chfm2
all_owners_parcelles_pd.head(2)

In [ ]:
# Save heatmap with prices for each parcelle
from heatmap import parcelles_prices
m = parcelles_prices(all_owners_parcelles_pd.dropna())
m.save('export/parcelles_prices.html')
m

The resulting map is accessible [here](export/parcelles_prices.html)

### 3.4 Price by quartier

We want to find out the median price for each quartier taking into account all prices of the parcelles predicted in the previous section.

For a correct rendering, we want to print only the quartiers from where we scraped the data. We therefore remove the two quartiers without data:

In [ ]:
# Remove '90 - Zones foraines' and '17 - Beaulieu / Grey / Boisy from quartiers
quartiers_to_drop = (quartiers_pd['Name'] == '17 - Beaulieu / Grey / Boisy') | \
                    (quartiers_pd['Name'] == '90 - Zones foraines')
scraped_quartiers_pd = quartiers_pd[~quartiers_to_drop].copy()

We compute now the media price for each quartier considering all parcelles. We consider the *median* and not the *mean* because there may be some outliers on our data since the data comes from different website and we don't have any certitude about the data pubblished.

In [ ]:
# for each parcelle, compute quartier
all_owners_parcelles_pd['quartier'] = \
    all_owners_parcelles_pd[['lat', 'long']] \
    .apply(lambda row: get_quartier(row['lat'], row['long']), axis='columns')

# group by quartier
price_by_quartier = all_owners_parcelles_pd[['CHF/m2', 'quartier']] \
                        .groupby('quartier').agg('median')
price_by_quartier

We can now merge the `scraped_quartiers_pd` that contains the geometry of the quartiers with `price_by_quartier`:

In [ ]:
# Merge scraped quartiers with price_by_quartier
scraped_quartiers_pd = scraped_quartiers_pd.merge(price_by_quartier, left_on='Name', right_index=True)
scraped_quartiers_pd.head(2)

In [ ]:
from heatmap import parcelles_prices_by_quartiers
m = parcelles_prices_by_quartiers(scraped_quartiers_pd)
m.save('export/parcelles_prices_by_quartiers.html')
m

The resulting map is accessible [here](export/parcelles_prices_by_quartiers.html)

From the resulting map, we can notice that quartiers close to the lake generally are more expensive. This is consistent with the conclusion we found in the 'linear regression by distance' section. 

### 3.5 Denoising owner type

We want to revisit the colorful owner types map by trying to find spatial clusters with a certain ownership type.
We will approach this problem as a denoising one, and we will attribute to each parcel an owner type which is given by the category the most represented in its local neighborhood.

For each parcel, we therefore find all the parcels within a distance R (the _radius_). We will calculate the prevalent owner type within that radius and reassign it as the parcel owner type.

In [ ]:
# The positions are in degrees. To accurately calculate distances we need to correct
# by the local linearization of the number of meters per degree of latitude/longitude in lausanne

EARTH_RADIUS_METERS = 6.3e6

# lausanne is approximately at pi/4 degrees north (46° something)
meters_per_easting_degree = 2*(EARTH_RADIUS_METERS*np.sin(np.pi/4))*np.pi/360
meters_per_northing_degree = 2*EARTH_RADIUS_METERS*np.pi/360

def compute_distance(delta_east, delta_north):
    """distance in meter given difference in coordinates"""
    return np.sqrt((delta_east*meters_per_easting_degree)**2 +\
        (delta_north*meters_per_northing_degree)**2)

def polygons_intersection(poly):
    """returns a function that gives the intersection areas of a serie of polygons with `poly`"""
    def inters(serie):
            return serie['geometry'].intersection(poly).area
    return inters

In [ ]:
# this cells takes a couple of minutes to run

RADIUS = 100

polygons_df = complete_owners_parcelles_pd[['parc_num', 'owner', 'geometry', 'owner_type']]

# cache this for performance purposes
polygons_df['x'] = polygons_df['geometry'].centroid.x
polygons_df['y'] = polygons_df['geometry'].centroid.y

# prepare the serie to store the results
own_types = polygons_df[['parc_num', 'owner_type']].copy().set_index('parc_num').sort_index()['owner_type']

# iterate over every parcel
for (idx, (parc_no, proprio, poly, owner_type, x, y)) in polygons_df.iterrows():
    distances = compute_distance(polygons_df['x'] - x, polygons_df['y'] - y)
    # select the cells whose center is closer than RADIUS to the current one
    neigh = distances[distances < RADIUS]
    
    # now weight each neighbour contribution by the size of their intersecting area with
    # the circle of radius RADIUS
    circle = poly.centroid.buffer(RADIUS/meters_per_easting_degree)
    neighbor_parcels = pd.concat((polygons_df, neigh), axis='columns', join='inner')
    neighbor_parcels['intersection'] = neighbor_parcels.apply(polygons_intersection(circle), axis='columns')
    intersection_per_cat = neighbor_parcels.groupby('owner_type')['intersection'].sum()
    own_types.loc[parc_no] = intersection_per_cat.sort_values().index[-1]
    
m = by_owners_category(complete_owners_parcelles_pd, own_types.to_frame())  
m.save("export/by_owners_category_denoised.html")

In [ ]:
own_types.to_frame().head(2)

The resulting map is accessible [here](export/by_owners_category_denoised.html)

### 3.6 Entropy map

To study local monopolies of ownership of the land, we produce an entropy map that will seek to detect areas where the owners in a local neighborhood of parcls are few.
To do this we select the nearest neighbors of each parcels, and calculate the Shannon entropy of the distribution of owners in this neighborhood.

In [ ]:
# this cells takes a couple of minutes to run

def polygons_intersection(poly):
    def inters(serie):
            return serie['geometry'].intersection(poly).area
    return inters

K = 10
RADIUS = 100

polygons_df = complete_owners_parcelles_pd[['parc_num', 'owner', 'geometry', 'owner_type']]

# cache this for performance purposes
polygons_df['x'] = polygons_df['geometry'].centroid.x
polygons_df['y'] = polygons_df['geometry'].centroid.y
# prepare the serie to store the results
entropy = polygons_df[['parc_num', 'owner']].copy().set_index('parc_num').sort_index()['owner']

for (idx, (parc_no, proprio, poly, owner_type, x, y)) in polygons_df.iterrows():
    distances = compute_distance(polygons_df['x'] - x, polygons_df['y'] - y)
    # select the cells whose center is closer than RADIUS to the current one
    neigh = distances.sort_values(ascending=True)[:K]
    # now weight each neighbour contribution by the size of their intersecting area with
    # the circle of radius RADIUS
#     circle = poly.centroid.buffer(RADIUS/meters_per_easting_degree)
    neighbor_parcels = pd.concat((polygons_df, neigh), axis='columns', join='inner')
#     neighbor_parcels['intersection'] = neighbor_parcels.apply(polygons_intersection(circle), axis='columns')
    intersection_per_owner = neighbor_parcels.groupby('owner').size()
    p = intersection_per_owner / intersection_per_owner.sum()
    entropy.loc[parc_no] = scipy.stats.entropy(p)

In [ ]:
from heatmap import entropy_owners
m = entropy_owners(complete_owners_parcelles_pd._to_geo(), entropy)
m.save("export/entropy_owners.html")
m

The resulting map is accessible [here](export/entropy_owners.html)

### 4. Data for website

Yet that we have all data, we can prepare ad-hoc maps to be posted on our final website.

The first map show owners by type of category:

In [ ]:
from heatmap import by_owners_all_in_one
m = by_owners_all_in_one(
    complete_owners_parcelles_pd._to_geo(),
    parcels_categories, own_types.to_frame(), quartiers_pd._to_geo())
m.save("export/by_owners_all_in_one.html")
m

The resulting map is accessible [here](export/by_owners_all_in_one.html)

In [ ]:
from heatmap import by_rents_all_in_one
m = by_rents_all_in_one(rent_prices_pd, scraped_quartiers_pd, all_owners_parcelles_pd.dropna())
m.save("export/by_rents_all_in_one.html")
m

The resulting map is accessible [here](export/by_rents_all_in_one.html)

# Conclusion

Our final goal has been to try to understand the relation between real estate owners and price. 

After scraping, parsing and cleaning the data from the cadastre of Lausanne and the rent prices of different rent portals, we went through different phase to try to find as many relations and links as possible. Muliple models such has linear regression or k-nearest-neighbour have been used to verify or prove our assumptions. 

From the most important results a data-driven story has born and can be found at the following address: [Who owns Lausanne?](https://who-owns-lausanne.github.io/).

Thanks for reading. 

Jonathan, Pietro and Yann.